# Probabilistic Bayesian Neural Networks in Tensorflow

## Introduction

Taking a probabilistic approach to deep learning allows to account for *uncertainty*,
so that models can assign less levels of confidence to incorrect predictions.
Sources of uncertainty can be found in the data, due to measurement error or
noise in the labels, or the model, due to insufficient data availability for
the model to learn effectively.

In [1]:
%%capture
!pip -Uqq install tensorflow-probability
!pip -Uqq install tensorflow-datasets

In [92]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

from dataclasses import dataclass
from tensorflow.keras import layers, Model, Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError

print("numpy                 ", np.__version__)
print("pandas                ", pd.__version__)
print("tensorflow            ", tf.__version__)
print("tensorflow-datasets   ", tfp.__version__)
print("tensorflow-probability", tfds.__version__)

numpy                  1.19.5
pandas                 1.1.5
tensorflow             2.4.0
tensorflow-datasets    0.12.1
tensorflow-probability 4.0.1


## The dataset
We use the Wine Quality dataset, which is available in the TensorFlow Datasets. We use the red wine subset, which contains 4,898 examples. The dataset has 11 numerical physicochemical features of the wine, and the task is to predict the wine quality, which is a score between 0 and 10.

In [62]:
@dataclass
class Config:
    DATASET_SIZE = 4898
    BATCH_SIZE = 256
    TRAIN_SIZE = int(DATASET_SIZE * 0.85)
    BASELINE_HIDDEN_UNITS = [8, 8]
    LEARNING_RATE = 0.001
    EPOCHS = 100

config = Config()

In [21]:
dataset = (
    tfds.load(name="wine_quality", as_supervised=True, split="train")
    .map(lambda x, y: (x, tf.cast(y, tf.float32)))
    .prefetch(buffer_size=config.DATASET_SIZE)
    .cache()
)

train_dataset = (
    dataset.take(config.TRAIN_SIZE).shuffle(buffer_size=config.TRAIN_SIZE).batch(config.BATCH_SIZE)
)
test_dataset = dataset.skip(config.TRAIN_SIZE).batch(config.BATCH_SIZE)

train_dataset.cardinality(), test_dataset.cardinality()

(<tf.Tensor: shape=(), dtype=int64, numpy=17>,
 <tf.Tensor: shape=(), dtype=int64, numpy=3>)

In [31]:
sample_record = next(iter(train_dataset))[0]
list(sample_record.keys())

['alcohol',
 'chlorides',
 'citric acid',
 'density',
 'fixed acidity',
 'free sulfur dioxide',
 'pH',
 'residual sugar',
 'sulphates',
 'total sulfur dioxide',
 'volatile acidity']

In [39]:
df = pd.DataFrame(sample_record)
df

,alcohol,chlorides,citric acid,density,fixed acidity,free sulfur dioxide,pH,residual sugar,sulphates,total sulfur dioxide,volatile acidity
0,8.7,0.057,0.49,1.00055,6.8,55.0,3.00,19.299999,0.56,247.0,0.240
1,9.3,0.035,0.37,0.99845,6.6,62.0,3.02,15.400000,0.40,153.0,0.220
2,9.2,0.168,0.40,0.99910,8.0,29.0,3.07,12.400000,0.64,190.0,0.430
3,9.5,0.049,0.26,0.99907,7.4,67.0,3.06,15.600000,0.68,210.0,0.330
4,8.8,0.055,0.24,0.99880,7.1,46.0,3.12,15.400000,0.49,198.0,0.350
...,...,...,...,...,...,...,...,...,...,...,...
251,9.5,0.050,0.49,0.99510,6.3,67.0,3.23,7.100000,0.34,210.0,0.230
252,11.2,0.036,0.32,0.99196,7.4,27.0,3.15,1.900000,0.49,119.0,0.360
253,10.1,0.046,0.18,0.99400,7.0,9.0,3.38,1.300000,0.47,62.0,0.240
254,10.1,0.045,0.30,0.99249,8.6,16.0,2.95,0.900000,0.39,109.0,0.310


In [40]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
alcohol,256.0,10.540394,1.218503,8.60000,9.50000,10.5000,11.3000,13.700000
chlorides,256.0,0.044117,0.017533,0.01400,0.03500,0.0415,0.0500,0.180000
citric acid,256.0,0.320625,0.110980,0.00000,0.26750,0.3000,0.3600,0.800000
density,256.0,0.993850,0.002917,0.98742,0.99143,0.9934,0.9958,1.000550
fixed acidity,256.0,6.838671,0.810536,4.90000,6.30000,6.8000,7.3000,9.200000
free sulfur dioxide,256.0,35.558594,15.769880,6.00000,24.75000,34.0000,47.0000,83.000000
pH,256.0,3.185156,0.142521,2.87000,3.08000,3.1800,3.2900,3.570000
residual sugar,256.0,6.037696,4.946777,0.80000,1.60000,4.5500,9.0750,19.799999
sulphates,256.0,0.495352,0.116723,0.29000,0.41000,0.4800,0.5500,0.940000
total sulfur dioxide,256.0,139.695312,40.554577,45.00000,112.00000,135.0000,169.2500,255.000000


## Experiment 1. Baseline Model

In [97]:
def create_model_inputs(feature_names):
    return {
        feature: Input(name=feature, shape=(1,), dtype=tf.float32)
        for feature in feature_names
    }

In [98]:
inputs = create_model_inputs(feature_names = sample_record.keys())
input_values = [value for _, value in sorted(inputs.items())]
features = layers.concatenate(input_values)
features = layers.BatchNormalization()(features)
for units in config.BASELINE_HIDDEN_UNITS:
    features = layers.Dense(units, activation="sigmoid")(features)
outputs = Dense(units=1)(features)
baseline_model = Model(inputs=inputs, outputs=outputs)


In [99]:
baseline_model.compile(
    optimizer=RMSprop(config.LEARNING_RATE),
    loss=MeanSquaredError(),
    metrics=[RootMeanSquaredError()],
)
baseline_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
alcohol (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
chlorides (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
citric acid (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
density (InputLayer)            [(None, 1)]          0                                            
____________________________________________________________________________________________

In [100]:
history = baseline_model.fit(train_dataset, epochs=config.EPOCHS, validation_data=test_dataset)

Epoch 1/100
17/17 [==============================] - 1s 14ms/step - loss: 34.1131 - root_mean_squared_error: 5.8406 - val_loss: 32.3752 - val_root_mean_squared_error: 5.6899
Epoch 2/100
17/17 [==============================] - 0s 7ms/step - loss: 31.9079 - root_mean_squared_error: 5.6487 - val_loss: 29.9390 - val_root_mean_squared_error: 5.4717
Epoch 3/100
17/17 [==============================] - 0s 7ms/step - loss: 30.0712 - root_mean_squared_error: 5.4837 - val_loss: 28.0993 - val_root_mean_squared_error: 5.3009
Epoch 4/100
17/17 [==============================] - 0s 7ms/step - loss: 28.3693 - root_mean_squared_error: 5.3263 - val_loss: 26.4272 - val_root_mean_squared_error: 5.1407
Epoch 5/100
17/17 [==============================] - 0s 8ms/step - loss: 26.7047 - root_mean_squared_error: 5.1676 - val_loss: 24.8318 - val_root_mean_squared_error: 4.9831
Epoch 6/100
17/17 [==============================] - 0s 8ms/step - loss: 25.4002 - root_mean_squared_error: 5.0397 - val_loss: 23.2908

In [69]:
_, train_rmse = baseline_model.evaluate(train_dataset, verbose=0)
_, test_rmse = baseline_model.evaluate(test_dataset, verbose=0)
print(f"Train RMSE {round(train_rmse, 3)}")
print(f"Test  RMSE {round(test_rmse, 3)}")

Train RMSE 0.76
Test  RMSE 0.752


In [91]:
samples, targets = next(iter(test_dataset))
predicted = baseline_model(samples)
for pred, targ in zip(predicted[:5], targets[:5]):
    print(f"Predicted: {pred[0]:.2} - Actual: {targ:.2}")

Predicted: 6.6 - Actual: 7.0
Predicted: 5.4 - Actual: 5.0
Predicted: 5.6 - Actual: 7.0
Predicted: 5.6 - Actual: 6.0
Predicted: 5.8 - Actual: 6.0


## Experiment 2: Bayesian neural network(BNN)

The object of the Bayesian approach for modeling neural networks is to capture
the *epistemic uncertainty*, which is uncertainty about the model fitness,
due to limited training data.

The idea is that, instead of learning specific weight (and bias) *values* in the
neural network, the Bayesian approach learns weight *distributions*
- from which we can sample to produce an output for a given input -
to encode weight uncertainty.

Thus, we need to define prior and the posterior distributions of these weights,
and the training process is to learn the parameters of these distributions.

Define the prior weight distribution as Normal of mean=0 and stddev=1.Note that, in this example, the we prior distribution is not trainable, as we fix its parameters.

In [104]:
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = Sequential([
        tfp.layers.DistributionLambda(
            lambda t: tfp.distributions.MultivariateNormalDiag(
                # normal of mean=0 and stddev=1
                loc=tf.zeros(n), scale_diag=tf.ones(n)
            )
        )
    ])
    return prior_model

Define variational posterior weight distribution as multivariate Gaussian. Note that the learnable parameters for this distribution are the means, variances, and covariances.

In [105]:
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = Sequential([
        tfp.layers.VariableLayer(
            tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
        ),
        tfp.layers.MultivariateNormalTriL(n)
    ])
    return posterior_model

We use the `tfp.layers.DenseVariational` layer instead of the standard
`keras.layers.Dense` layer in the neural network model.

In [108]:
inputs = create_model_inputs(feature_names=sample_record.keys())
features = layers.concatenate(list(inputs.values()))
features = layers.BatchNormalization()(features)

features = tfp.layers.DenseVariational(
    units=config.BASELINE_HIDDEN_UNITS[0],
    make_prior_fn=prior,
    make_posterior_fn=posterior,
    kl_weight=1 / config.TRAIN_SIZE,
    activation="sigmoid",
)(features)
outputs = Dense(units=1)(features)
bnn_model_small = Model(inputs=inputs, outputs=outputs)
bnn_model_small.compile(
    optimizer=RMSprop(learning_rate=config.LEARNING_RATE),
    loss=MeanSquaredError(),
    metrics=[RootMeanSquaredError()],
)

In [114]:
num_epochs = 500
train_sample_size = int(config.TRAIN_SIZE * 0.3)
small_train_dataset = (
    train_dataset.unbatch().take(train_sample_size).batch(config.BATCH_SIZE)
)

print("Start training the model...")
bnn_model_small.fit(small_train_dataset, epochs=num_epochs, validation_data=test_dataset, verbose=0)
print("Model training finished.")
_, rmse = bnn_model_small.evaluate(small_train_dataset, verbose=0)
print(f"Train RMSE: {round(rmse, 3)}")

print("Evaluating model performance...")
_, rmse = bnn_model_small.evaluate(test_dataset, verbose=0)
print(f"Test RMSE: {round(rmse, 3)}")



Start training the model...
Model training finished.
Train RMSE: 0.744
Evaluating model performance...
Test RMSE: 0.744
